# Task 3: Helper notebook for loading the data and saving the predictions

In [ ]:
import pickle
import gzip
import numpy as np
import os
from fastcore.all import *

data_path = Path('../data/processed/')


### Helper functions

In [ ]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [ ]:
def save_zipped_pickle(obj, filename):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, 2)

### Load data, make predictions and save prediction in correct format

In [ ]:
# load data
train_data = load_zipped_pickle(data_path/"train.pkl")
test_data = load_zipped_pickle(data_path/"test.pkl")
samples = load_zipped_pickle(data_path/"sample.pkl")

In [ ]:
train_data[2]["video"][:,:,0]
expert = [list for list in train_data if not (list["dataset"] == "amateur")]


In [ ]:
import matplotlib.pyplot as plt

def show_mitral_segmentation(expert_id, expert):

    print("Expert #", expert_id)

    frameid_1 = expert[expert_id]['frames'][0]
    frameid_2 = expert[expert_id]['frames'][len(expert[expert_id]['frames'])-1]

    # Plotting two frames side by side
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    # Plotting the first frame
    frame1 = expert[expert_id]['video'][:, :, frameid_1]
    label1 = expert[expert_id]['label'][:, :, frameid_1]

    axs[0].imshow(frame1, cmap='gray')
    axs[0].imshow(label1, cmap='jet', alpha=0.5)  # Overlay label1 with transparency
    axs[0].set_title('Frame '+str(frameid_1))

    # Plotting the second frame
    frame2 = expert[expert_id]['video'][:, :, frameid_2]
    label2 = expert[expert_id]['label'][:, :, frameid_2]

    axs[1].imshow(frame2, cmap='gray')
    axs[1].imshow(label2, cmap='jet', alpha=0.5)  # Overlay label2 with transparency
    axs[1].set_title('Frame 2'+str(frameid_2))

    plt.show()


In [ ]:
expert_id=1

In [ ]:
show_mitral_segmentation(expert_id, expert)
expert_id+=1

In [ ]:
# make prediction for test
predictions = []
for d in test_data:
    prediction = np.array(np.zeros_like(d['video']), dtype=np.bool)
    height = prediction.shape[0]
    width = prediction.shape[1]
    prediction[int(height/2)-50:int(height/2+50), int(width/2)-50:int(width/2+50)] = True
    
    # DATA Strucure
    predictions.append({
        'name': d['name'],
        'prediction': prediction
        }
    )

In [ ]:
13+15+20+9+14+4+12

In [ ]:
# save in correct format
save_zipped_pickle(predictions, 'my_predictions.pkl')

### TASK 3: MITRAL VALVE SEGMENTATION

The mitral valve is the largest valve of the heart and regulates the blood flow between the left atrium and the left ventricle. It is composed by two leaflets, the anterior and the posterior leaflet, that are attached to a fibrous ring known as the mitral annulus. In healthy patients, the left atrium contracts during diastole and the blood flows through the open MV into the left ventricle while it is dilating.
Echocardiography is a medical imaging technique that produces 2D pictures and videos of the heart using ultrasound waves generated by a transducer, scattered and reflected by biological tissues and read by a detector. Echocardiography is the standard imaging tool in the clinical routine to perform the diagnosis of most of heart diseases and dysfunctions, including MV diseases.
Mitral valve segmentation specifies a crucial first step to establish a machine learning pipeline that can support practitioners in performing multiple tasks including the diagnosis of mitral valve diseases, surgical planning, and intraoperative procedures. Therefore, this task will be concerned with the segmentation of the mitral valve in echocardiography videos.

The training set consists of 65 videos in which the mitral valve is labled in three different frames in each video. Additionllay, a bounding box surounding the mitral valve is provided for each video. The test set consists of 20 videos. For each video in the test set the mitral valve should be segmented for all frames.

### DATA DESCRIPTION


The samples originate from two different data sets: One data set was labeled by experienced cardiologist and consists of high resolution videos, and the other was labeled by amateurs and has relatively low resolution. While the training data consists of videos from both data sets the test videos (where you need to make the predictions) are evaluated based on expert labels only. It is up to you to decide to which extend you want to use the amateur labeled videos for the training of your model.

The data for this task consists of the following files:

train.pkl - the training data with the videos, bounding boxes, labels and information about the data source
test.pkl - the test data with the videos where the MV should be segmented
sample.pkl - a sample submission file in the correct format
task3.ipynb - a helper notebook containing functions for reading and saving the pickle (*.pkl) files
The training data (train.pkl) is a list of dictionaries containing the following keys: name, video, box, label, frames, dataset. An single dictionary has the following strucure:

    {
     'name': 'AC8ASN45B9',

     'video': array([...], dtype='uint8'),

     'box': array([...], dtype='bool'),

     'label': array([...], dtype='bool'),

     'frames': [41, 65, 167],

     'dataset': 'amateur'
    }
  
The shape of the video is (height, width, number of frames), the box has shape (height, width) and the label has the same shape as the video. The frames is a list of indices (starting at zero), indicating which frames are labeled. The dataset key describes if the video was labeled by amateurs or experts.